**Análise sobre os dados de Lesão Dolosa 2014 - 2024**

## Bibliotecas utilizadas

In [6]:
# Bibliotecas para
import pandas as pd
import numpy as np

# Bibliotecas para vizualização de dados
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import folium

import geopandas as gpd


# Bibliotecas para importação e exportação de arquivos
from google.colab import files
import json

import unicodedata

In [57]:
!pip install --upgrade plotly

In [4]:
!pip install unidecode
from unidecode import unidecode

  Using cached Unidecode-1.3.8-py3-none-any.whl.metadata (13 kB)
Using cached Unidecode-1.3.8-py3-none-any.whl (235 kB)


## Criação/junção dos Dataframes


In [ ]:
#@title Criação dos dataframes que serão utilizados
#@markdown O código abaixo não precisa ser rodado. Estou deixando no notebook apenas para ficar registrado.

#@markdown Os dataframes disponibilzados pelo SSP-SP são anuais, nesse sentido, visando fazer uma análise temporal mais completa, irei juntar os dataframes de 2014 a 2024. Após isso  farei o download do df concatenado para facilitar a importação dos dados.

#@markdown Também criei um novo dataframe com algumas informações do IBGE

def concatenar_dfs_anos_diferentes(df_previous, df_current, year):
    columns = [f'{month} {year}' for month in ['Janeiro', 'Fevereiro', 'Marco', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']] + ['Total']

    df_filtered = df_current[columns].copy()

    df_filtered.rename(columns={'Total': f'Total {year}'}, inplace=True)

    df_combined = pd.concat([df_previous, df_filtered], axis=1)

    return df_combined


anos = range(2014, 2025)
dfs = [pd.read_csv(f'lesao_corporal_dolosa_{ano}.csv') for ano in anos]

df_combined = dfs[0]

for ano, df_current in zip(anos[1:], dfs[1:]):
    df_combined = concatenar_dfs_anos_diferentes(df_combined, df_current, ano)

df_combined = df_combined.rename(columns= {'Total': 'Total 2014'})
df_combined

df_combined.to_csv('lesao_dolosa_2014_2024.csv', index=False)
files.download('lesao_dolosa_2014_2024.csv')


"""
O código abaixo também não precisa ser rodado. Eu fiz a junção de vários dataframes do ibge com
aglumas informações sobre os municípios para realizar a análise de influência das variáveis nos
casos de crime
"""

df_pop = pd.read_csv('/content/populacao.csv')
df_dens = pd.read_csv('/content/densidade_demografica.csv')
df_sal = pd.read_csv('/content/salario_medio_mensal.csv')
df_esc = pd.read_csv('/content/taxa_de_escolarizacao_6_a_14_anos_2010.csv')
df_arbo = pd.read_csv('/content/arborizacao_vias_pub.csv')
df_aurb = pd.read_csv('/content/area_urbanizada.csv')
df_esg = pd.read_csv('/content/egotamento_sanitario_adequ.csv')
df_pib = pd.read_csv('/content/pib_pc.csv')
df_urbv = pd.read_csv('/content/urbanizacao_vias_pub.csv')



df = pd.merge(df_pop, df_dens, on='Local')
df = pd.merge(df, df_sal, on='Local')
df = pd.merge(df, df_esc, on='Local')
df = pd.merge(df, df_arbo, on='Local')
df = pd.merge(df, df_aurb, on='Local')
df = pd.merge(df, df_esg, on='Local')
df = pd.merge(df, df_pib, on='Local')
df = pd.merge(df, df_urbv, on='Local')


lista_municipios = df_combined['municipios'].unique()

df_limpo = df[df['Local'].isin(lista_municipios)]

df_limpo.to_csv('dados_municipios.csv', index=False)
files.download('dados_municipios.csv')



## Lista que serão utilizadas para faciliatar as consultas

In [9]:
# lista com nome dos municipios
lista_municipios = ['Aguas de Lindoia', 'Amparo', 'Atibaia', 'Bom Jesus dos Perdoes',
                    'Braganca Paulista', 'Cabreuva', 'Campinas',
                    'Campo Limpo Paulista', 'Estiva Gerbi', 'Holambra', 'Indaiatuba',
                    'Itapira', 'Itatiba', 'Itupeva', 'Jaguariuna', 'Jarinu',
                    'Joanopolis', 'Jundiai', 'Lindoia', 'Louveira', 'Mogi Guacu',
                    'Monte Alegre do Sul', 'Morungaba', 'Nazare Paulista', 'Paulinia',
                    'Pedra Bela', 'Pedreira', 'Pinhalzinho', 'Piracaia', 'Serra Negra',
                    'Socorro', 'Tuiuti', 'Valinhos', 'Vargem', 'Varzea Paulista',
                    'Vinhedo']

In [10]:
# lista de anos
anos = range(2014, 2025)

In [11]:
# lista de meses
meses = ['Janeiro', 'Fevereiro', 'Marco', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']

## Funções adicionais que serão utilizadas ao longo das análises

In [12]:
# Função para remover acentos
def remover_acentos(texto):
    return ''.join(
        char for char in unicodedata.normalize('NFKD', texto)
        if not unicodedata.combining(char)
    )

## Importação dos Dataframes criados

In [155]:
df_combined = pd.read_csv('/content/lesao_dolosa_2014_2024.csv')
df_ibge = pd.read_csv('/content/dados_municipios.csv')


# arquivo de regiões imediatas
regioes_imediatas = gpd.read_file("/content/RG2017_rgi.shp.csv")
geojson_regioes = json.loads(regioes_imediatas.to_json())


# Abrindo arquivo GeoJson municípios
with open('/content/geojs-35-mun.json', 'r', encoding='utf-8') as f:
    geojson_municipios = json.load(f)

In [156]:
# Vizualização df_combined
df_combined.head(3)

,municipios,cod_ibge,reg_administrativa,reg_metropolitanas,Janeiro 2014,Fevereiro 2014,Marco 2014,Abril 2014,Maio 2014,Junho 2014,...,Abril 2024,Maio 2024,Junho 2024,Julho 2024,Agosto 2024,Setembro 2024,Outubro 2024,Novembro 2024,Dezembro 2024,Total 2024
0,Amparo,3501905,RA de Campinas,Demais municípios,43,34,33,26,14,21,...,19,18,11,21,0,0,0,0,0,130
1,Atibaia,3504107,RA de Campinas,Demais municípios,58,52,69,42,56,55,...,42,49,48,39,0,0,0,0,0,317
2,Bom Jesus dos Perdões,3507100,RA de Campinas,Demais municípios,8,5,3,5,10,6,...,3,4,5,5,0,0,0,0,0,32


In [157]:
# Vizualização df_ibge
df_ibge.head(3)

,Local,"""População no último censo""","""Densidade demográfica""","""Salário médio mensal dos trabalhadores formais""","""Taxa de escolarização de 6 a 14 anos de idade""","""Arborização de vias públicas""","""Área urbanizada""","""Esgotamento sanitário adequado""","""PIB per capita""","""Urbanização de vias públicas"""
0,Águas de Lindóia,17930,298.21,1.8,96.2,82.1,8.53,92.9,30971.23,73.3
1,Amparo,68008,152.72,2.6,95.7,83.9,21.31,88.7,70437.56,63.8
2,Atibaia,158647,331.54,2.6,97.3,67.0,64.28,80.8,57512.36,33.7


In [21]:
geojson_municipios['features'][0]

{'type': 'Feature',
 'properties': {'id': '3500105',
  'name': 'Adamantina',
  'description': 'Adamantina'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-51.0578686748, -21.3988835488],
    [-51.0536454977, -21.4046228386],
    [-51.0450043489, -21.4037238276],
    [-51.045439432, -21.4063589008],
    [-51.0397224364, -21.4067760488],
    [-51.0330880762, -21.408863112],
    [-51.0247100808, -21.4208663742],
    [-51.0245845013, -21.4263462599],
    [-51.0033347502, -21.4385388237],
    [-50.9925874583, -21.4367999015],
    [-50.9911744426, -21.4418090183],
    [-50.9772464698, -21.4526305135],
    [-50.9674002039, -21.4750044331],
    [-50.9701364325, -21.4791879543],
    [-50.9651317696, -21.4838230857],
    [-50.9672264355, -21.4912547321],
    [-50.9644621647, -21.5045943621],
    [-50.9684617375, -21.5150857178],
    [-50.9639572448, -21.5215964229],
    [-50.9656099653, -21.5382668569],
    [-50.96736118, -21.5431056598],
    [-50.9739312904, -21.5459116694],
    [-50.97

In [22]:
geojson_regioes

{'attrlabl/_Sync': {'0': 'TRUE', '1': 'TRUE', '2': 'TRUE', '3': 'TRUE'},
 'attrlabl/__text': {'0': 'FID', '1': 'Shape', '2': 'rgi', '3': 'nome_rgi'},
 'attalias/_Sync': {'0': 'TRUE', '1': 'TRUE', '2': 'TRUE', '3': 'TRUE'},
 'attalias/__text': {'0': 'FID', '1': 'Shape', '2': 'rgi', '3': 'nome_rgi'},
 'attrtype/_Sync': {'0': 'TRUE', '1': 'TRUE', '2': 'TRUE', '3': 'TRUE'},
 'attrtype/__text': {'0': 'OID',
  '1': 'Geometry',
  '2': 'String',
  '3': 'String'},
 'attwidth/_Sync': {'0': 'TRUE', '1': 'TRUE', '2': 'TRUE', '3': 'TRUE'},
 'attwidth/__text': {'0': '4', '1': '0', '2': '50', '3': '100'},
 'atprecis/_Sync': {'0': 'TRUE', '1': 'TRUE', '2': 'TRUE', '3': 'TRUE'},
 'atprecis/__text': {'0': '0', '1': '0', '2': '0', '3': '0'},
 'attscale/_Sync': {'0': 'TRUE', '1': 'TRUE', '2': 'TRUE', '3': 'TRUE'},
 'attscale/__text': {'0': '0', '1': '0', '2': '0', '3': '0'},
 'attrdef/_Sync': {'0': 'TRUE', '1': 'TRUE', '2': '', '3': ''},
 'attrdef/__text': {'0': 'Internal feature number.',
  '1': 'Feature

## Análise exploratória

#### Nesta seção, primeiramente será feita uma EDA separada para cada base importada neste notebook. Após isso, quando oportuno, realizarei análises que envolvem mais de uma base

### Casos

In [158]:
# removendo o acento de àguas de lindoia
df_combined['municipios'] = df_combined['municipios'].apply(unidecode)

df_combined.sort_values(by='municipios', inplace=True)

df_combined.head(3)

,municipios,cod_ibge,reg_administrativa,reg_metropolitanas,Janeiro 2014,Fevereiro 2014,Marco 2014,Abril 2014,Maio 2014,Junho 2014,...,Abril 2024,Maio 2024,Junho 2024,Julho 2024,Agosto 2024,Setembro 2024,Outubro 2024,Novembro 2024,Dezembro 2024,Total 2024
35,Aguas de Lindoia,3500501,RA de Campinas,Demais municípios,3,6,13,8,7,5,...,7,9,8,5,0,0,0,0,0,47
0,Amparo,3501905,RA de Campinas,Demais municípios,43,34,33,26,14,21,...,19,18,11,21,0,0,0,0,0,130
1,Atibaia,3504107,RA de Campinas,Demais municípios,58,52,69,42,56,55,...,42,49,48,39,0,0,0,0,0,317


In [159]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, 35 to 33
Columns: 147 entries, municipios to Total 2024
dtypes: int64(144), object(3)
memory usage: 41.6+ KB


In [160]:
df_combined.shape

(36, 147)

In [161]:
df_combined.dtypes

,0
municipios,object
cod_ibge,int64
reg_administrativa,object
reg_metropolitanas,object
Janeiro 2014,int64
...,...
Setembro 2024,int64
Outubro 2024,int64
Novembro 2024,int64
Dezembro 2024,int64


In [162]:
df_combined.describe()

,cod_ibge,Janeiro 2014,Fevereiro 2014,Marco 2014,Abril 2014,Maio 2014,Junho 2014,Julho 2014,Agosto 2014,Setembro 2014,...,Abril 2024,Maio 2024,Junho 2024,Julho 2024,Agosto 2024,Setembro 2024,Outubro 2024,Novembro 2024,Dezembro 2024,Total 2024
count,3.600000e+01,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,...,36.000000,36.000000,36.000000,36.000000,36.0,36.0,36.0,36.0,36.0,36.000000
mean,3.529700e+06,33.805556,34.611111,31.638889,29.333333,26.916667,29.305556,26.777778,26.111111,30.527778,...,28.666667,26.777778,22.000000,23.944444,0.0,0.0,0.0,0.0,0.0,184.416667
std,1.733350e+04,71.366587,66.453970,60.975476,59.704989,50.382749,57.018202,54.642794,52.037227,60.653812,...,56.120788,55.845763,44.419429,48.655021,0.0,0.0,0.0,0.0,0.0,367.815045
min,3.500501e+06,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,...,1.000000,2.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,10.000000
25%,3.520146e+06,4.000000,5.000000,5.750000,4.750000,5.750000,6.000000,5.000000,4.000000,4.750000,...,5.750000,4.000000,4.750000,2.750000,0.0,0.0,0.0,0.0,0.0,32.000000
50%,3.527158e+06,17.000000,15.000000,13.500000,10.500000,12.500000,13.000000,12.500000,12.000000,13.500000,...,12.000000,9.000000,9.000000,10.500000,0.0,0.0,0.0,0.0,0.0,75.500000
75%,3.538303e+06,32.250000,37.750000,33.250000,28.750000,29.000000,29.500000,27.000000,22.750000,28.500000,...,29.000000,22.250000,22.750000,24.250000,0.0,0.0,0.0,0.0,0.0,178.250000
max,3.557303e+06,427.000000,393.000000,359.000000,355.000000,299.000000,335.000000,325.000000,305.000000,356.000000,...,330.000000,330.000000,266.000000,285.000000,0.0,0.0,0.0,0.0,0.0,2173.000000


In [163]:
#@title Série temporal de casos totais ao longo dos anos
anos = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

totals = []

for ano in anos:
    total = df_combined[f'Total {ano}'].sum()
    totals.append(total)

fig = px.line(x=anos, y=totals, title='Total de casos ao longo dos anos')

fig.update_layout(xaxis_title='Anos', yaxis_title='Numero de casos')

fig.show()


In [164]:
#@title Série temporal de casos totais ao longo dos anos e meses

total_meses = []
datas = []


for ano in anos:
    for mes in meses:
        total_mes = df_combined[f'{mes} {ano}'].sum()
        total_meses.append(total_mes)
        datas.append(f'{ano}-{meses.index(mes)+1:02}')

fig = px.line(x=datas, y=total_meses, title='Total de casos ao longo dos meses e anos')

fig.update_layout(xaxis_title='Datas', yaxis_title='Numero de casos')

fig.show()


In [165]:
#@title Boxplot casos X meses do ano
fig = go.Figure()

for ano in anos:
    for mes in meses:
        fig.add_trace(go.Box(y=df_combined[f'{mes} {ano}'], name=f'{mes} {ano}', visible=False))

for trace in fig.data[:len(meses)]:
    trace.visible = True

sliders = [
    {
        "active": 0,
        "currentvalue": {"prefix": "Ano: "},
        "pad": {"b": 10},
        "steps": [
            {
                "label": str(ano),
                "method": "update",
                "args": [
                    {"visible": [False] * len(fig.data)},
                    {"title": f"Boxplot meses do ano {ano}"}
                ]
            }
            for ano in range(2014, 2025)
        ]
    }
]

for i, step in enumerate(sliders[0]['steps']):
    visible = [False] * len(fig.data)
    visible[i * len(meses):(i + 1) * len(meses)] = [True] * len(meses)
    step['args'][0]['visible'] = visible

fig.update_layout(
    sliders=sliders,
    title='Boxplot meses do ano 2014',
    yaxis_title='Número de casos'
)

fig.show()

In [166]:
#@title Histograma de casos por ano por município
fig = make_subplots(rows=6, cols=6, subplot_titles=df_combined['municipios'].unique())

subplot_counter = 0

for municipio in df_combined['municipios'].unique():
    valores = []

    for ano in anos:
        valor = df_combined[df_combined['municipios'] == municipio][f'Total {ano}'].values[0]
        valores.append(valor)

    row = subplot_counter // 6 + 1
    col = subplot_counter % 6 + 1

    fig.add_trace(
        go.Bar(x=anos, y=valores, name=municipio),
        row=row, col=col
    )

    subplot_counter += 1

fig.update_layout(
    title_text="Total de Casos por Ano para Cada Município",
    height=1000, width=1700,
    showlegend=False
)

fig.show()

In [169]:
#@title Variação percentual anual de casos por município

for municipio in df_combined['municipios'].unique():
    variacao = []

    for ano in anos[:-1]:
        total_ano_anterior = df_combined[df_combined['municipios'] == municipio][f'Total {ano}'].values[0]
        total_ano_atual = df_combined[df_combined['municipios'] == municipio][f'Total {ano+1}'].values[0]

        variacao_ano = ((total_ano_atual - total_ano_anterior) / total_ano_anterior)
        variacao.append(variacao_ano)

    variacoes_municipio[municipio] = variacao

variacoes_municipio_df = pd.DataFrame(variacoes_municipio).transpose()

variacoes_municipio_df.columns = [f'{ano}-{ano+1}' for ano in anos[:-1]]

variacoes_municipio_df.head()

,2014-2015,2015-2016,2016-2017,2017-2018,2018-2019,2019-2020,2020-2021,2021-2022,2022-2023,2023-2024
Aguas de Lindoia,0.103448,-0.062500,-0.200000,0.083333,-0.615385,0.000000,-0.166667,0.880000,0.382979,-0.276923
Amparo,-0.209302,0.088235,-0.142857,-0.085586,0.162562,-0.381356,0.130137,0.175758,0.087629,-0.383886
Atibaia,-0.085496,0.073456,-0.118196,-0.089947,-0.003876,-0.173152,0.007059,0.065421,0.149123,-0.395038
Bom Jesus dos Perdoes,0.318182,-0.034483,0.071429,-0.122222,-0.253165,0.101695,-0.061538,0.065574,0.107692,-0.555556
Braganca Paulista,-0.045819,-0.100840,-0.058745,-0.134752,-0.067213,-0.214411,0.020134,0.188596,0.025830,-0.402878


In [168]:
variacoes_municipio_df.describe()

,2014-2015,2015-2016,2016-2017,2017-2018,2018-2019,2019-2020,2020-2021,2021-2022,2022-2023,2023-2024
count,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000
mean,-0.073803,0.047248,-0.047212,-0.032019,-0.042968,-0.128022,-0.001795,0.150762,0.145374,-0.404187
std,0.173980,0.166713,0.183551,0.210291,0.202613,0.171090,0.170158,0.309806,0.229470,0.094193
min,-0.500000,-0.243243,-0.551724,-0.500000,-0.615385,-0.381356,-0.370370,-0.533333,-0.216216,-0.655172
25%,-0.152322,-0.065237,-0.153712,-0.122679,-0.165452,-0.230218,-0.097973,0.031216,0.026728,-0.452596
50%,-0.086118,0.023773,-0.068148,-0.017523,-0.012296,-0.156786,0.011968,0.137616,0.095973,-0.405581
75%,0.054286,0.114370,0.062821,0.086029,0.060784,-0.030659,0.066720,0.226841,0.208617,-0.378307
max,0.318182,0.500000,0.384615,0.408163,0.555556,0.360000,0.480000,1.117647,1.142857,-0.189474


### Dados do IBGE

#### Unidades dados IBGE:



*   População no último censo:
*   Densidade demogŕafica: hab/km²
*   Salário médio mensal dos trabalahadores formais: salários mínimos
*   Taxa de escolarização de 6 a 14 anos de idade: %
*   Arborização de vias públicas: %
*   Área urbanizada: km²
*   Esgotamento sanitário adequado: %
*   PIB per capita: reais
*   Urbanização de vias públicas: %



In [170]:
#@title Df IBGE
df_ibge = df_ibge.rename(columns={'Local' : 'municipios'})
df_ibge['municipios'] = df_ibge['municipios'].apply(unidecode)


nomes_colunas = ['municipios', 'Populacao_ultimo_censo', 'Densidade_demografica',
                   'Salario_medio_mensal_trabalhadores_formais', 'Taxa_escolarizacao_6_14_anos',
                   'Arborizacao_vias_publicas', 'Area_urbanizada', 'Esgotamento_sanitario_adequado',
                   'PIB_per_capita', 'Urbanizacao_vias_publicas']


df_ibge = df_ibge.rename(columns=dict(zip(df_ibge.columns, nomes_colunas)))

df_ibge.head(3)

,municipios,Populacao_ultimo_censo,Densidade_demografica,Salario_medio_mensal_trabalhadores_formais,Taxa_escolarizacao_6_14_anos,Arborizacao_vias_publicas,Area_urbanizada,Esgotamento_sanitario_adequado,PIB_per_capita,Urbanizacao_vias_publicas
0,Aguas de Lindoia,17930,298.21,1.8,96.2,82.1,8.53,92.9,30971.23,73.3
1,Amparo,68008,152.72,2.6,95.7,83.9,21.31,88.7,70437.56,63.8
2,Atibaia,158647,331.54,2.6,97.3,67.0,64.28,80.8,57512.36,33.7


In [35]:
df_ibge.describe()

,Populacao_ultimo_censo,Densidade_demografica,Salario_medio_mensal_trabalhadores_formais,Taxa_escolarizacao_6_14_anos,Arborizacao_vias_publicas,Area_urbanizada,Esgotamento_sanitario_adequado,PIB_per_capita,Urbanizacao_vias_publicas
count,3.600000e+01,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000
mean,1.018585e+05,444.527500,2.650000,97.450000,76.877778,30.281389,81.505556,79973.650833,52.097222
std,1.975670e+05,606.706736,0.707713,0.791202,18.329868,43.801026,17.071513,96399.783500,17.726034
min,6.557000e+03,34.240000,1.600000,95.700000,27.900000,1.620000,38.000000,14148.820000,8.800000
25%,1.519150e+04,97.262500,2.075000,97.150000,67.000000,6.412500,73.500000,29089.530000,39.650000
50%,4.506150e+04,196.065000,2.600000,97.600000,81.250000,18.350000,88.650000,45799.135000,54.200000
75%,1.118455e+05,513.932500,3.200000,98.000000,90.425000,34.670000,94.300000,89088.827500,64.075000
max,1.139047e+06,3296.440000,4.600000,98.800000,100.000000,245.140000,97.600000,457517.700000,82.700000


### Merge casos + ibge

In [36]:
#@title combined + ibge
df = pd.merge(df_combined, df_ibge, on='municipios')

df.head()

,municipios,cod_ibge,reg_administrativa,reg_metropolitanas,Janeiro 2014,Fevereiro 2014,Marco 2014,Abril 2014,Maio 2014,Junho 2014,...,Total 2024,Populacao_ultimo_censo,Densidade_demografica,Salario_medio_mensal_trabalhadores_formais,Taxa_escolarizacao_6_14_anos,Arborizacao_vias_publicas,Area_urbanizada,Esgotamento_sanitario_adequado,PIB_per_capita,Urbanizacao_vias_publicas
0,Aguas de Lindoia,3500501,RA de Campinas,Demais municípios,3,6,13,8,7,5,...,47,17930,298.21,1.8,96.2,82.1,8.53,92.9,30971.23,73.3
1,Amparo,3501905,RA de Campinas,Demais municípios,43,34,33,26,14,21,...,130,68008,152.72,2.6,95.7,83.9,21.31,88.7,70437.56,63.8
2,Atibaia,3504107,RA de Campinas,Demais municípios,58,52,69,42,56,55,...,317,158647,331.54,2.6,97.3,67.0,64.28,80.8,57512.36,33.7
3,Bom Jesus dos Perdoes,3507100,RA de Campinas,Demais municípios,8,5,3,5,10,6,...,32,22006,203.07,2.6,97.7,96.5,8.42,96.3,55196.42,74.2
4,Braganca Paulista,3507605,RA de Campinas,Demais municípios,83,86,73,69,56,55,...,332,176811,344.94,2.5,97.0,71.2,59.52,87.1,46153.09,56.5


### Choropleth



In [37]:
#@title Casos per capita
resultados = []

for municipio in df['municipios'].unique():
    for ano in anos:
        casos = df[df['municipios'] == municipio][f'Total {ano}'].values[0]
        populacao = df[df['municipios'] == municipio]['Populacao_ultimo_censo'].values[0]
        cod_ibge = df[df['municipios'] == municipio]['cod_ibge'].values[0]

        casos_pc = casos / populacao
        resultados.append({
            'Cod_ibge' : cod_ibge,
            'municipios': municipio,
            'ano': ano,
            'casos_per_capita': casos_pc
        })

df_casos_pc = pd.DataFrame(resultados)

# df_casos_pc['casos_per_capita'].describe()

# df_casos_pc.head()


df_map = df_casos_pc.pivot_table(index=['Cod_ibge', 'municipios'],
                            columns='ano',
                            values='casos_per_capita',
                            fill_value=0).reset_index()
df_map.head(3)

ano,Cod_ibge,municipios,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,3500501,Aguas de Lindoia,0.004852,0.005354,0.005020,0.004016,0.004350,0.001673,0.001673,0.001394,0.002621,0.003625,0.002621
1,3501905,Amparo,0.004426,0.003500,0.003808,0.003264,0.002985,0.003470,0.002147,0.002426,0.002853,0.003103,0.001912
2,3504107,Atibaia,0.004129,0.003776,0.004053,0.003574,0.003253,0.003240,0.002679,0.002698,0.002874,0.003303,0.001998


In [58]:
#@title criação do mapa
# Abrindo arquivo GeoJson municípios
with open('/content/geojs-35-mun.json', 'r', encoding='utf-8') as f:
    geojson_municipios = json.load(f)


# arquivo de regiões imediatas
regioes_imediatas = gpd.read_file("/content/RG2017_rgi.shp.csv")
geojson_regioes = json.loads(regioes_imediatas.to_json())


"""
O nome dos municípios no GeoJson possui acentos, portanto, ao checar a quantidade de valores
no GeoJson filtrado aparecem apenas 24 features ao invés de 36, pois a lista_municipios não
possui acento então alguns valores não dão match

len(filtered_geojson['features'])

"""

# removendo acentos
for feature in geojson_municipios['features']:
    feature['properties']['name'] = unidecode(feature['properties']['name'])


"""
O arquivo contem dados de todos os municípios de São Paulo, portanto,
farei um filtro para analisar apenas os municípios de interesse
"""

filtered_geojson = {'type': 'FeatureCollection', 'features': []}
for feature in geojson_municipios['features']:
    if feature['properties']['name'] in lista_municipios:
        filtered_geojson['features'].append(feature)


# camada do mapa de regiões imediatas
fig.add_trace(go.Choroplethmapbox(
    geojson=geojson_regioes,
    locations=regioes_imediatas.index,
    featureidkey="properties.id",
    z=[0] * len(regioes_imediatas),
    colorscale="Blues",
    colorbar_title="Regiões Imediatas",
    showscale=False,
    marker_opacity=0.8
))

fig = go.Figure()

for ano in anos:

    fig.add_trace(go.Choroplethmapbox(
        geojson=filtered_geojson,
        locations=df_map['Cod_ibge'],
        featureidkey="properties.id",
        z=df_map[ano],
        colorscale="Viridis",
        colorbar_title="Casos per capita",
        zmin=df_map[ano].min(),
        zmax=df_map[ano].max(),
        marker_opacity=0.6,
        visible=False,
        hovertemplate="<b>Município:</b> %{properties.name}<br>" +
                  "<b>Casos per capita:</b> %{z:.5f}<extra></extra>"
    ))

fig.data[0].visible = True

steps = []
for i, ano in enumerate(anos):
    step = dict(
        method="update",   # método para atualizar a vizualização
        args=[{"visible": [j == i for j in range(len(fig.data))]},
              {"title": f"Distribuição de {ano} e Regiões Imediatas"}],
        label=str(ano)
    )
    steps.append(step)

sliders = [dict(
    active=0,
    steps=steps,
    currentvalue={"prefix": "Ano: "},
    pad={"t": 10, "b": 10}, # t = top, b = bellow. para ajustar o espaçamento t e b
    len=0.6,
    ticklen = 3,
    x=0.24
)]

fig.update_layout(
    sliders=sliders,
    mapbox_style="open-street-map",
    mapbox=dict(
        center={"lat": -23.55, "lon": -46.64},
        zoom=5.5
    ),
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    title=f"Distribuição de {anos[0]} e Regiões Imediatas"
)

fig.show()

#Análise de séries temporais

### Gerar séries para os outros crimes e buscar tendências e sazonalidades em comum

# Análise de regressão